In [ ]:
import openai
import os
from langchain.embeddings.openai import OpenAIEmbeddings
# openai.api_key = '824fe43e851f4862af326fa83c3d3cfe'
# openai.api_base =  'https://mtcaichat01.openai.azure.com' 
# openai.api_type = 'azure' # Necessary for using the OpenAI library with Azure OpenAI
# openai.api_version = '2023-03-15-preview' # This likely will change with future releases

os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_BASE"] = "https://mtcaichat01.openai.azure.com"
os.environ["OPENAI_API_KEY"] = "824fe43e851f4862af326fa83c3d3cfe"
os.environ["OPENAI_API_VERSION"] = "2023-03-15-preview"

print(os.environ["OPENAI_API_TYPE"])
print(os.environ["OPENAI_API_BASE"])
print(os.environ["OPENAI_API_KEY"])
print(os.environ["OPENAI_API_VERSION"])

deployment_name = 'embeddingada002' # SDK calls this "engine", but naming



In [ ]:
from langchain.text_splitter import CharacterTextSplitter

with open('CauseAndEffectOfHomelessness.txt') as f:
    text = f.read()

text_splitter = CharacterTextSplitter(chunk_size=5, chunk_overlap=0)
text = text_splitter.split_text(text)
text

In [ ]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import AzureOpenAI
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma

embeddings = OpenAIEmbeddings()
vectorstore = Chroma.from_texts(text, embeddings)
chain = RetrievalQA.from_chain_type(llm = AzureOpenAI(
    deployment_name="embeddingada002",
    model_name="text-embedding-ada-002",
))

In [ ]:
query = "Where does homeless person stays?"
chain.run(query)

In [ ]:
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chat_models import AzureChatOpenAI
from langchain.schema import HumanMessage

BASE_URL = "https://mtcaichat01.openai.azure.com"
API_KEY = "824fe43e851f4862af326fa83c3d3cfe"
DEPLOYMENT_NAME = "gpt354k"
llm = AzureOpenAI(
    openai_api_base=BASE_URL,
    openai_api_version="2023-03-15-preview",
    deployment_name=DEPLOYMENT_NAME,
    openai_api_key=API_KEY,
    openai_api_type = "azure",
)
#Proof that LLM is working without Indexing/embedding features.
print("----------------------------")
print(llm("Tell me a joke"))
print("----------------------------")

loader = TextLoader('CauseAndEffectOfHomelessness.txt', encoding='utf8')
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)
embeddings = OpenAIEmbeddings()
db = Chroma.from_documents(texts, embeddings)
retriever = db.as_retriever()
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)
query = "What did the president say about Ketanji Brown Jackson"
qa.run(query)